In [1]:
import pandas as pd
import numpy as np
import plotnine as p9
from scipy import stats
from common import load_data, pareto_rank
import os
import json

from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from mlxtend.frequent_patterns import fpgrowth, fpmax
import pysubgroup as ps

# Subgroup Discovery

Combinations of input + configuration features that go hand in hand

- category=X -> foob=2

Constant configuration features and their options

- cabac=True

For variable configuration features -> which input features describe which option to pick

- blabla = 1 and foob = 2 if resolution < 900 and height > 200
- blabla = 0 else

- foob = 2
- foob = 3

In [2]:
random_state = 1234
test_size = 0.2
pareto_cutoff = 0.4
(
    perf_matrix,
    input_features,
    config_features,
    all_performances,
    input_preprocessor,
    config_preprocessor,
) = load_data(system="x264", data_dir="../data", input_properties_type="tabular")

# Normalization is needed for the Pareto cutoff
# We can normalize before splitting, because
# we normalize per input and we also split per input.
# There is no data leakage.
normalized_metrics = (
    perf_matrix[["inputname"] + all_performances]
    .groupby("inputname", as_index=False)
    .transform(lambda x: (x - x.min()) / (x.max() - x.min()))
)
cutoff_mask = (normalized_metrics <= pareto_cutoff).all(axis=1)

nmdf = perf_matrix[["inputname"] + all_performances].groupby("inputname", as_index=True).transform(lambda x: (x - x.min()) / (x.max() - x.min()))
perf_matrix = pd.merge(perf_matrix, nmdf, suffixes=("_raw", None), left_index=True, right_index=True)

all_perf_raw = [f"{p}_raw" for p in all_performances]
all_perf_norm = [f"{p}" for p in all_performances]

train_inp, test_inp = train_test_split(
    perf_matrix["inputname"].unique(),
    test_size=test_size,
    random_state=random_state,
)
train_perf = perf_matrix[perf_matrix.inputname.isin(train_inp)]
test_perf = perf_matrix[perf_matrix.inputname.isin(test_inp)]

all_performances

['fps', 'etime', 'cpu', 'size', 'kbs']

In [3]:
performances = ["fps", "cpu"]
performances = ["size", "etime"]
icm = (
    train_perf[["inputname", "configurationID"] + performances]
    .sort_values(["inputname", "configurationID"])
    .set_index(["inputname", "configurationID"])
)
icm_ranked_measures = icm.groupby(
    "inputname"
).transform(  # Go from measured values to ranks within each input group
    lambda x: stats.rankdata(x, method="min")
)

# TODO Adjust pareto_rank with cutoff
# 1. Second version that ranks dominated configurations by their "dominators"
# 2. 
icm["ranks"] = icm.groupby("inputname", group_keys=False).apply(pareto_rank)

# Calculate the Pareto ranks for the test data
icm_test = (
    test_perf[["inputname", "configurationID"] + performances]
    .sort_values(["inputname", "configurationID"])
    .set_index(["inputname", "configurationID"])
)
icm_test["ranks"] = icm_test.groupby("inputname", group_keys=False).apply(pareto_rank)

icm

size     etime  ranks
inputname            configurationID                           
Animation_1080P-01b3 1                0.070142  0.003932      2
                     2                0.029658  0.066841      5
                     3                0.001764  0.090433      5
                     4                0.008462  0.098296     19
                     5                0.006305  0.094364     16
...                                        ...       ...    ...
Vlog_720P-6d56       197              0.141975  0.723088     68
                     198              0.074812  0.583593     22
                     199              0.088498  0.836276     38
                     200              0.088498  0.809277     38
                     201              0.209511  0.752510    116

[206829 rows x 3 columns]

In [4]:
subdf = icm[icm.ranks <= 1]
subdf

size     etime  ranks
inputname            configurationID                           
Animation_1080P-01b3 12               0.067026  0.001311      1
                     14               0.070142  0.000000      1
                     19               0.058674  0.011796      1
                     58               0.003181  0.039318      1
                     68               0.000770  0.070773      1
...                                        ...       ...    ...
Vlog_720P-6d56       64               0.079784  0.021115      1
                     67               0.079784  0.025268      1
                     71               0.003678  0.027345      1
                     72               0.000000  0.033922      1
                     79               0.084493  0.010730      1

[15038 rows x 3 columns]

In [5]:
subdf = icm.join(input_features).join(config_features).reset_index()
subdf["Target"] = subdf.ranks <= 1
subdf

dataset = subdf[list(input_features.columns) + list(config_features.columns) + ["Target"]]
dataset

,resolution,WIDTH,HEIGHT,SPATIAL_COMPLEXITY,TEMPORAL_COMPLEXITY,CHUNK_COMPLEXITY_VARIATION,COLOR_COMPLEXITY,category,cabac,ref,...,me,direct,deblock,b_adapt,b_pyramid,open_gop,rc_lookahead,scenecut,weightb,Target
0,1080,1920,1080,0.098,0.004,0.017,0.005,Animation,0,1,...,dia,NaN,0:0:0,NaN,NaN,NaN,NaN,0.0,NaN,False
1,1080,1920,1080,0.098,0.004,0.017,0.005,Animation,1,1,...,dia,auto,1:0:0,1.0,2.0,0.0,NaN,40.0,1.0,False
2,1080,1920,1080,0.098,0.004,0.017,0.005,Animation,1,1,...,hex,auto,1:0:0,1.0,2.0,0.0,10.0,40.0,1.0,False
3,1080,1920,1080,0.098,0.004,0.017,0.005,Animation,1,2,...,hex,auto,1:0:0,1.0,2.0,0.0,20.0,40.0,1.0,False
4,1080,1920,1080,0.098,0.004,0.017,0.005,Animation,1,2,...,hex,auto,1:0:0,1.0,2.0,0.0,30.0,40.0,1.0,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
206824,720,1280,720,1.074,0.428,11.794,0.415,Vlog,1,16,...,tesa,spatial,1:0:0,2.0,2.0,0.0,60.0,40.0,1.0,False
206825,720,1280,720,1.074,0.428,11.794,0.415,Vlog,1,16,...,hex,spatial,1:0:0,2.0,2.0,0.0,60.0,40.0,1.0,False
206826,720,1280,720,1.074,0.428,11.794,0.415,Vlog,1,16,...,tesa,spatial,1:0:0,2.0,2.0,0.0,60.0,40.0,1.0,False
206827,720,1280,720,1.074,0.428,11.794,0.415,Vlog,1,16,...,tesa,spatial,1:0:0,2.0,2.0,0.0,60.0,40.0,1.0,False


In [6]:
target = ps.BinaryTarget ('Target', True)
searchspace = ps.create_selectors(dataset, nbins=10, ignore=['Target', 'configurationID'])
task = ps.SubgroupDiscoveryTask (
    dataset,
    target,
    searchspace,
    result_set_size=100,
    depth=10,
    qf=ps.WRAccQF())
result = ps.DFS().execute(task)
rdf = result.to_dataframe()

In [15]:
rdf  #.sort_values("target_share_sg", ascending=False)

,quality,subgroup,size_sg,size_dataset,positives_sg,positives_dataset,size_complement,relative_size_sg,relative_size_complement,coverage_sg,coverage_complement,target_share_sg,target_share_complement,target_share_dataset,lift
0,0.036034,chroma_qp_offset==0 AND fast_pskip==1 AND trel...,77175,206829,13064,15038,129654,0.373134,0.626866,0.868733,0.131267,0.169278,0.015225,0.072707,2.328203
1,0.036034,chroma_qp_offset==0 AND fast_pskip==1 AND qpma...,77175,206829,13064,15038,129654,0.373134,0.626866,0.868733,0.131267,0.169278,0.015225,0.072707,2.328203
2,0.034909,fast_pskip==1 AND trellis==0,85407,206829,13430,15038,121422,0.412935,0.587065,0.893071,0.106929,0.157247,0.013243,0.072707,2.162738
3,0.034909,fast_pskip==1 AND qpmax==69 AND trellis==0,85407,206829,13430,15038,121422,0.412935,0.587065,0.893071,0.106929,0.157247,0.013243,0.072707,2.162738
4,0.034679,chroma_qp_offset==0 AND trellis==0,87465,206829,13532,15038,119364,0.422886,0.577114,0.899854,0.100146,0.154713,0.012617,0.072707,2.127889
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,0.028899,chroma_qp_offset==0 AND fast_pskip==1 AND me==...,40131,206829,8895,15038,166698,0.194030,0.805970,0.591502,0.408498,0.221649,0.036851,0.072707,3.048508
96,0.028879,mbtree==0 AND rc_lookahead.isnull() AND trelli...,44247,206829,9190,15038,162582,0.213930,0.786070,0.611118,0.388882,0.207698,0.035970,0.072707,2.856624
97,0.028879,mbtree==0 AND qpmax==69 AND rc_lookahead.isnul...,44247,206829,9190,15038,162582,0.213930,0.786070,0.611118,0.388882,0.207698,0.035970,0.072707,2.856624
98,0.028804,fast_pskip==1 AND rc_lookahead.isnull() AND tr...,42189,206829,9025,15038,164640,0.203980,0.796020,0.600146,0.399854,0.213918,0.036522,0.072707,2.942181


In [8]:
dataset = subdf.join(input_features).reset_index()[list(input_features.columns) + ["configurationID"]]

all_results = []

for c in dataset.configurationID.unique():
    dataset["Target"] = dataset["configurationID"] == c

    target = ps.BinaryTarget ('Target', True)
    searchspace = ps.create_selectors(dataset, ignore=['Target', 'configurationID'])
    task = ps.SubgroupDiscoveryTask (
        dataset,
        target,
        searchspace,
        result_set_size=20,
        depth=10,
        qf=ps.SimpleBinomialQF())
    result = ps.DFS().execute(task)
    rdf = result.to_dataframe()
    rdf["target"] = c
    all_results.append(rdf)

ValueError: columns overlap but no suffix specified: Index(['resolution', 'WIDTH', 'HEIGHT', 'SPATIAL_COMPLEXITY',
       'TEMPORAL_COMPLEXITY', 'CHUNK_COMPLEXITY_VARIATION', 'COLOR_COMPLEXITY',
       'category'],
      dtype='object')

In [ ]:
rdf = pd.concat(all_results)
rdf  #.sort_values("target_share_sg", ascending=False)

,quality,subgroup,size_sg,size_dataset,positives_sg,positives_dataset,size_complement,relative_size_sg,relative_size_complement,coverage_sg,coverage_complement,target_share_sg,target_share_complement,target_share_dataset,lift,target
0,0.006354,COLOR_COMPLEXITY: [0.09:0.18[ AND TEMPORAL_COM...,2,10221,1,468,10219,0.000196,0.999804,0.002137,0.997863,0.500000,0.045699,0.045788,10.919872,12
1,0.006354,COLOR_COMPLEXITY: [0.09:0.18[ AND SPATIAL_COMP...,2,10221,1,468,10219,0.000196,0.999804,0.002137,0.997863,0.500000,0.045699,0.045788,10.919872,12
2,0.006354,COLOR_COMPLEXITY: [0.09:0.18[ AND SPATIAL_COMP...,2,10221,1,468,10219,0.000196,0.999804,0.002137,0.997863,0.500000,0.045699,0.045788,10.919872,12
3,0.006354,COLOR_COMPLEXITY: [0.09:0.18[ AND HEIGHT>=1080...,2,10221,1,468,10219,0.000196,0.999804,0.002137,0.997863,0.500000,0.045699,0.045788,10.919872,12
4,0.006354,COLOR_COMPLEXITY: [0.09:0.18[ AND HEIGHT>=1080...,2,10221,1,468,10219,0.000196,0.999804,0.002137,0.997863,0.500000,0.045699,0.045788,10.919872,12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15,0.002740,CHUNK_COMPLEXITY_VARIATION<0.67 AND COLOR_COMP...,13,10221,1,1,10208,0.001272,0.998728,1.000000,0.000000,0.076923,0.000000,0.000098,786.230769,128
16,0.002740,CHUNK_COMPLEXITY_VARIATION<0.67 AND COLOR_COMP...,13,10221,1,1,10208,0.001272,0.998728,1.000000,0.000000,0.076923,0.000000,0.000098,786.230769,128
17,0.002740,CHUNK_COMPLEXITY_VARIATION<0.67 AND COLOR_COMP...,13,10221,1,1,10208,0.001272,0.998728,1.000000,0.000000,0.076923,0.000000,0.000098,786.230769,128
18,0.002740,CHUNK_COMPLEXITY_VARIATION<0.67 AND COLOR_COMP...,13,10221,1,1,10208,0.001272,0.998728,1.000000,0.000000,0.076923,0.000000,0.000098,786.230769,128


In [ ]:
dataset[result.to_descriptions()[0][1].covers(dataset)]

,resolution,WIDTH,HEIGHT,SPATIAL_COMPLEXITY,TEMPORAL_COMPLEXITY,CHUNK_COMPLEXITY_VARIATION,COLOR_COMPLEXITY,category,configurationID,Target
9041,480,720,1080,0.293,0.03,0.568,0.122,VerticalVideo,15,False
9042,480,720,1080,0.293,0.03,0.568,0.122,VerticalVideo,25,False
9043,480,720,1080,0.293,0.03,0.568,0.122,VerticalVideo,26,False
9044,480,720,1080,0.293,0.03,0.568,0.122,VerticalVideo,45,False
9045,480,720,1080,0.293,0.03,0.568,0.122,VerticalVideo,46,False
9046,480,720,1080,0.293,0.03,0.568,0.122,VerticalVideo,47,False
9047,480,720,1080,0.293,0.03,0.568,0.122,VerticalVideo,62,False
9048,480,720,1080,0.293,0.03,0.568,0.122,VerticalVideo,80,False
9049,480,720,1080,0.293,0.03,0.568,0.122,VerticalVideo,81,False
9050,480,720,1080,0.293,0.03,0.568,0.122,VerticalVideo,84,False


In [ ]:
result.to_descriptions()[0][1]

(SPATIAL_COMPLEXITY<0.628 and WIDTH: [640:850[ and category=='VerticalVideo' and resolution==480)